# EASY 

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import recall_score
from sklearn.svm import SVC
import pandas as pd

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [62]:
df = pd.read_pickle("../data/train.pickle")
ln = pd.read_csv("../data/label_names.csv")
label_names = {}
for index, row in ln.iterrows():
    label_names[row['ClassId']] = row["SignName"]
df.keys()

dict_keys(['coords', 'labels', 'features', 'sizes'])

In [63]:
x_train, x_test, y_train, y_test = train_test_split(df["features"], df["labels"], train_size=0.8)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
correct_sz = x_train.shape[1] * x_train.shape[2] * x_train.shape[3]

x_train = x_train.reshape((x_train.shape[0], correct_sz))
x_test = x_test.reshape((x_test.shape[0], correct_sz))
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(27839, 32, 32, 3) (6960, 32, 32, 3) (27839,) (6960,)
(27839, 3072) (6960, 3072) (27839,) (6960,)


In [64]:
model = LogisticRegression(max_iter=500)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
error_LR = recall_score(y_test, y_pred, average="micro")
error_LR

/home/gabik/anaconda3/envs/hse_ml_course_2024/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=500)

Выбираем метрику полноты, потому что нам самое важное - положительные срабатывания 

In [66]:
res = {}
cnt = {}
for i in range(len(y_pred)):
    res[y_test[i]] = res.get(y_test[i], 0) + (y_test[i] == y_pred[i])
    cnt[y_test[i]] = cnt.get(y_test[i], 0 ) + 1
total = []
for key, count in cnt.items():
    total.append((res[key]/count, key, label_names[key]))
total.sort()
print("Хуже всего распознаются знаки: ", total[0][2])

Хуже всего распознаются знаки:  Bicycles crossing


## Medium

In [69]:
total_score = {"логистическая регрессия": error_LR}

In [70]:
model = KNeighborsClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
total_score["KNN"] = recall_score(y_test, y_pred, average="micro")

In [73]:
model = GaussianNB()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
total_score["НБК"] = recall_score(y_test, y_pred, average="micro")

In [79]:
kernels = ["linear", "poly", "rbf"]
for kernel in kernels:
    model = SVC(kernel=kernel)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    total_score["SVC. kernel=" + kernel] = recall_score(y_test, y_pred, average="micro")

In [82]:
total_score

{'логистическая регрессия': 0.9564655172413793,
 'KNN': 0.881896551724138,
 'НБК': 0.16982758620689656,
 'SVC linear': 0.962212643678161,
 'SVC poly': 0.7205459770114943,
 'SVC rbf': 0.8866379310344827}

In [81]:
best = (0, None)
worst = (2, None)
for key, val in total_score.items():
    if val > best[0]:
        best = (val, key)
    if val < worst[0]:
        worst = (val, key)
print(f"Лучше всего отработала модель {best[1]}, с показателем {best[0]}")
print(f"Хуже всего отработала модель {worst[1]}, с показателем {worst[0]}")

Лучше всего отработала модель SVC linear, с показателем 0.962212643678161
Хуже всего отработала модель НБК, с показателем 0.16982758620689656
